![Logo 1](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech1.jpg)
<div class="alert alert-block alert-info">
<h1> Komputerowe wspomaganie tłumaczenia </h1>
<h2> 2. <i>Zaawansowane użycie pamięci tłumaczeń</i> [laboratoria]</h2> 
<h3>Rafał Jaworski (2021)</h3>
</div>

![Logo 2](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech2.jpg)

Wiemy już, do czego służy pamięć tłumaczeń. Spróbujmy przeprowadzić mały research, którego celem będzie odkrycie, w jaki sposób do pamięci tłumaczeń podchodzą najwięksi producenci oprogramowania typu CAT.


### Ćwiczenie 1: Wykonaj analizę funkcjonalności pamięci tłumaczeń w programach SDL Trados Studio 2021 oraz Kilgray memoQ. Dla obu programów wypisz funkcje, które są związane z TM oraz zaznacz, które funkcje są wspólne dla obu programów oraz których funkcji Tradosa brakuje w memoQ oraz odwrotnie.

Odpowiedź: Obydwa programy (Trados i memoQ) oferują podobne funkcje związane z TM, takie jak tworzenie i zarządzanie bazami TM, importowanie/eksportowanie pamięci oraz automatyczne uzupełnianie segmentów podczas tłumaczenia. Główne różnice pojawiają się w funkcjonalności zaawansowanego zarządzania TM – Trados oferuje możliwość konsolidacji TM z różnych projektów oraz obsługę wielu baz TM jednocześnie, czego brakuje w memoQ. Z kolei memoQ zapewnia bardziej zaawansowane narzędzia do zarządzania kategoriami TM oraz umożliwia bardziej elastyczną konsolidację TM. Funkcja ICE (In-Context Exact match) jest dostępna w obu programach, umożliwiając uzyskanie najbardziej precyzyjnych dopasowań z pełnym uwzględnieniem kontekstu tekstu.

 Rozważmy przykładową pamięć tłumaczeń z poprzednich zajęć (można do niej dorzucić kilka przykładów):

In [10]:
translation_memory = [
                      ('Wciśnij przycisk Enter', 'Press the ENTER button'), 
                      ('Sprawdź ustawienia sieciowe', 'Check the network settings'),
                      ('Drukarka jest wyłączona', 'The printer is switched off'),
                      ('Wymagane ponowne uruchomienie komputera', 'System restart required')
                     ]

### Ćwiczenie 2: Zaimplementuj funkcję ice_lookup, przyjmującą trzy parametry: aktualnie tłumaczone zdanie, poprzednio tłumaczone zdanie, następne zdanie do tłumaczenia. Funkcja powinna zwracać dopasowania typu ICE. Nie pozwól, aby doszło do błędów podczas sprawdzania pierwszego i ostatniego przykładu w pamięci (ze względu na brak odpowiednio poprzedzającego oraz następującego przykładu).

In [3]:
def ice_lookup(sentence, prev_sentence, next_sentence):
    translation_memory = [
        ('Wciśnij przycisk Enter', 'Press the ENTER button'),
        ('Sprawdź ustawienia sieciowe', 'Check the network settings'),
        ('Drukarka jest wyłączona', 'The printer is switched off'),
        ('Wymagane ponowne uruchomienie komputera', 'System restart required')
    ]
    
    matches = []
    for i, (source, target) in enumerate(translation_memory):
        if source == sentence:
            prev_match = (i > 0 and translation_memory[i - 1][0] == prev_sentence) if prev_sentence else True
            next_match = (i < len(translation_memory) - 1 and translation_memory[i + 1][0] == next_sentence) if next_sentence else True
            
            if prev_match and next_match:
                matches.append((source, target))
    
    return matches

# Przykładowe wywołanie
current_sentence = 'Sprawdź ustawienia sieciowe'
previous_sentence = 'Wciśnij przycisk Enter'  # Poprzednie zdanie
next_sentence = 'Drukarka jest wyłączona'  # Następne zdanie

matches = ice_lookup(current_sentence, previous_sentence, next_sentence)
print(matches)


[('Sprawdź ustawienia sieciowe', 'Check the network settings')]


Inną powszechnie stosowaną techniką przeszukiwania pamięci tłumaczeń jest tzw. **fuzzy matching**. Technika ta polega na wyszukiwaniu zdań z pamięci, które są tylko podobne do zdania tłumaczonego. Na poprzednich zajęciach wykonywaliśmy funkcję tm_lookup, która pozwalała na różnicę jednego słowa.

Zazwyczaj jednak funkcje fuzzy match posiadają znacznie szersze możliwości. Ich działanie opiera się na zdefiniowaniu funkcji $d$ dystansu pomiędzy zdaniami $x$ i $y$. Matematycznie, funkcja dystansu posiada następujące właściwości:
1. $\forall_{x,y} d(x,y)\geqslant 0$
2. $\forall_{x,y} d(x,y)=0 \Leftrightarrow x=y$
3. $\forall_{x,y} d(x,y)=d(y,x)$
4. $\forall_{x,y,z} d(x,y) + d(y,z)\geqslant d(x,z)$

Rozważmy następującą funkcję:

In [7]:
def sentence_distance(x,y):
    return abs(len(y) - len(x))

### Ćwiczenie 3: Czy to jest poprawna funkcja dystansu? Które warunki spełnia?

Odpowiedz: Nie jest to poprawna funkcja dystansu, poniewaz skupia się ona na policzeniu wartosci bezwzględnej z róznicy długości dwóch zdań (ale mierzoną w liczbie znaków bo len(x) zwróci ile znaków miało słowo/zdanie x).

A teraz spójrzmy na taką funkcję:

In [4]:
def sentence_distance(x,y):
    if (x == y):
        return 0
    else:
        return 3

### Ćwiczenie 4: Czy to jest poprawna funkcja dystansu? Które warunki spełnia?

Odpowiedz: Warunek 1 jest zawsze spełniony bo w 1 lub 2 opcji zawsze dostaniemy >= 0. 2 warunek tez jest spelniony (wynika z pierwszego if). 3 warunek jets spelniony poniewaz jesli x!=y to i tak zawsze otrzymamy 3 (return 3). Warunek 4 tez jest spelniony bo: 0+0 >= 0, 0+3 >=3, 3+0>=3, 3+3 >= 3. Zatem jest to poprawna funkcja dystansu.

Wprowadźmy jednak inną funkcję dystansu - dystans Levenshteina. Dystans Levenshteina pomiędzy dwoma łańcuchami znaków definiuje się jako minimalną liczbę operacji edycyjnych, które są potrzebne do przekształcenia jednego łańcucha znaków w drugi. Wyróżniamy trzy operacje edycyjne:
* dodanie znaku
* usunięcie znaku
* zamiana znaku na inny

### Ćwiczenie 5: Czy dystans Levenshteina jest poprawną funkcją dystansu? Uzasadnij krótko swoją odpowiedź sprawdzając każdy z warunków.

Odpowiedź: Warunek 1 jest spelniony, bo edycja znakow to zawsze bedzie liczba nieujemna (>=0). Warunek 2 bedzie tez spelniony, poniewaz jesli nie zedytujemy znaku to dystans = 0, bo x=y. Warunek 3 tez jest spelniony, poniewaz do edycji x w y potrzebujemy tyle samo operacji co do zmienienia y w x. Warunek 4 jest spełniony, suma minimalnych operacji z x->y i z y->z może być równa liczbie operacji z x->z, ale niekoniecznie musi, ponieważ możliwa jest również inna, szybsza ścieżka. Zatem warunek nierówności trójkąta jest spełniony. Więc dystans Levenshteina jest poprawną funkcją dystansu.

W Pythonie dostępna jest biblioteka zawierająca implementację dystansu Levenshteina. Zainstaluj ją w swoim systemie przy użyciu polecenia:

`pip3 install python-Levenshtein`

I wypróbuj:

In [4]:
from Levenshtein import distance as levenshtein_distance

levenshtein_distance("kotek", "kotki")


2

Funkcja ta daje nam możliwość zdefiniowania podobieństwa pomiędzy zdaniami:

In [5]:
def levenshtein_similarity(x,y):
    return 1 - levenshtein_distance(x,y) / max(len(x), len(y))

Przetestujmy ją!

In [6]:
levenshtein_similarity('Program jest uruchomiony', 'Program jest uruchamiany')

0.9166666666666666

In [7]:
levenshtein_similarity('Spróbuj wyłączyć i włączyć komputer', 'Spróbuj włączyć i wyłączyć komputer')

0.9428571428571428

In [9]:
levenshtein_similarity('Spróbuj wyłączyć i włączyć komputer', 'Nie próbuj wyłączać i włączać drukarki')

0.631578947368421

### Ćwiczenie 6: Napisz funkcję fuzzy_lookup, która wyszuka w pamięci tłumaczeń wszystkie zdania, których podobieństwo Levenshteina do zdania wyszukiwanego jest większe lub równe od ustalonego progu.

In [ ]:
def fuzzy_lookup(sentence, threshold):
    for existing_sentence in translation_memory:
        if levenshtein_similarity(sentence, existing_sentence[0]) >= threshold:
            #print(sentence, existing_sentence)
            return [sentence, existing_sentence]
        
fuzzy_lookup('Wymagane ponowne uruchomienie monitora', 0.5)

Wymagane ponowne uruchomienie monitora ('Wymagane ponowne uruchomienie komputera', 'System restart required')


['Wymagane ponowne uruchomienie monitora',
 ('Wymagane ponowne uruchomienie komputera', 'System restart required')]